In [1]:
import coremltools
from src.datasets import camvid
from src.tiramisu import tiramisu

In [2]:
# the location to load weights from
weights = 'models/Tiramisu45-CamVid11-fine.h5'
# the size to crop images to
image_size = (352, 480)

# Class Labels

In [3]:
camvid11 = camvid.CamVid(
    mapping=camvid.CamVid.load_mapping(),
    target_size=(360, 480),
    crop_size=image_size,
    horizontal_flip=True,
    ignored_labels=['Void'],
)

In [4]:
camvid11.discrete_to_rgb_map

{0: (0, 128, 192),
 1: (128, 0, 0),
 2: (64, 0, 128),
 3: (192, 192, 128),
 4: (64, 64, 128),
 5: (64, 64, 0),
 6: (128, 64, 128),
 7: (0, 0, 192),
 8: (192, 128, 128),
 9: (128, 128, 128),
 10: (192, 192, 0),
 11: (0, 0, 0)}

# Model

In [5]:
# build the model for the image shape and number of labels
model = tiramisu.tiramisu((*image_size, 3), camvid11.n,
    class_weights=camvid11.class_mask,
    growth_rate=12,
    layer_sizes=[3, 3, 3, 3, 3],
    bottleneck_size=3,
)
model.load_weights(weights)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tiramisu_input (InputLayer)     (None, 352, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 352, 480, 48) 1344        Tiramisu_input[0][0]             
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 352, 480, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 352, 480, 48) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# CoreML Tools

In [6]:
# help(coremltools.converters.keras.convert)

In [7]:
coreml_model = coremltools.converters.keras.convert(model,
	input_names="image",
	image_input_names="image",
	image_scale=1/255.0,
)

0 : Tiramisu_input, <keras.engine.topology.InputLayer object at 0x10fe84f60>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x10fea5f28>
2 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0x10fea5a90>
3 : activation_1, <keras.layers.core.Activation object at 0x10fea5a58>
4 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x10fec6e10>
5 : concatenate_1, <keras.layers.merge.Concatenate object at 0x10ff60860>
6 : batch_normalization_2, <keras.layers.normalization.BatchNormalization object at 0x10ff60f28>
7 : activation_2, <keras.layers.core.Activation object at 0x10ff60b38>
8 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x119ff4dd8>
9 : concatenate_2, <keras.layers.merge.Concatenate object at 0x11a0daba8>
10 : batch_normalization_3, <keras.layers.normalization.BatchNormalization object at 0x11a072320>
11 : activation_3, <keras.layers.core.Activation object at 0x11a0726d8>
12 : conv2d_4, <keras.layers.convolutional.Conv2D obj

104 : concatenate_25, <keras.layers.merge.Concatenate object at 0x11b6948d0>
105 : batch_normalization_25, <keras.layers.normalization.BatchNormalization object at 0x11b6946a0>
106 : activation_25, <keras.layers.core.Activation object at 0x11b6d3320>
107 : conv2d_26, <keras.layers.convolutional.Conv2D object at 0x11b73f898>
108 : concatenate_26, <keras.layers.merge.Concatenate object at 0x11b7c2630>
109 : batch_normalization_26, <keras.layers.normalization.BatchNormalization object at 0x11b7c24a8>
110 : activation_26, <keras.layers.core.Activation object at 0x11b7c2ac8>
111 : conv2d_27, <keras.layers.convolutional.Conv2D object at 0x11b852b70>
112 : concatenate_27, <keras.layers.merge.Concatenate object at 0x11b8b1b38>
113 : conv2d_transpose_2, <keras.layers.convolutional.Conv2DTranspose object at 0x11b8b1fd0>
114 : concatenate_28, <keras.layers.merge.Concatenate object at 0x11b963f28>
115 : batch_normalization_27, <keras.layers.normalization.BatchNormalization object at 0x11b963eb8>
1

In [8]:
coreml_model.author = 'Kautenja'
coreml_model.license = 'MIT'
coreml_model.short_description = '45 Layers Tiramisu Semantic Segmentation Model trained on CamVid 11.'
coreml_model.input_description['image'] = 'Input image in RGB order'
coreml_model.output_description['output1'] = 'Softmax output from the network'

In [9]:
coreml_model.save('models/Tiramisu45.mlmodel')